In [16]:
import pandas as pd
import os

# 1. Configuración de nombres de archivo y columnas
archivo_a = '/media/PORT-DISK/Practicas/MicroBleeds_Generation/ADNI_original_dataset_downloaded/MAYOADIRL_MRI_MCH_12Feb2026.csv'
archivo_b = '/media/PORT-DISK/Practicas/MicroBleeds_Generation/ADNI_original_dataset_downloaded/idaSearch_2_19_2026.csv'

columna_id_a = 'LONI_IMG_ID' 
columna_id_b = 'Image ID'

# 2. Carga de datos con detección de separador
try:
    df1 = pd.read_csv(archivo_a, sep=None, engine='python')
    df2 = pd.read_csv(archivo_b, sep=None, engine='python')
    
    # Limpiamos posibles espacios en los nombres de las columnas (para evitar el KeyError)
    df1.columns = df1.columns.str.strip()
    df2.columns = df2.columns.str.strip()
    
    print("Archivos cargados. Columnas en B detectadas:", df2.columns.tolist())
    
except Exception as e:
    print(f"Error al cargar archivos: {e}")
    raise

# 3. NORMALIZACIÓN DE IDs (Agregado por mí)
# En el archivo A: Convertimos a string, quitamos la 'I' del principio y limpiamos espacios
df1[columna_id_a] = df1[columna_id_a].astype(str).str.replace(r'^I', '', regex=True).str.strip()

# En el archivo B: Aseguramos que sea string y limpiamos espacios
df2[columna_id_b] = df2[columna_id_b].astype(str).str.strip()

# 4. Realizar el Join (Cruce)
df_inner = pd.merge(df1, df2, left_on=columna_id_a, right_on=columna_id_b, how='inner')

# 5. Definir filtros de limpieza
# Filtro A: TYPE MCH y STATUS Definite
condicion_mch = (df_inner['TYPE'] == 'MCH') & (df_inner['STATUS'] == 'Definite')

# Filtro B: TYPE en blanco (nulo o vacío)
# .isna() detecta NaNs, y el chequeo de string vacío detecta celdas sin texto
condicion_sanos = (df_inner['TYPE'].isna()) | (df_inner['TYPE'].astype(str).str.strip() == '') | (df_inner['TYPE'].astype(str) == 'nan')

# Aplicar el filtrado final: Solo nos quedamos con una de estas dos opciones
df_filtrado = df_inner[condicion_mch | condicion_sanos].copy()

# 6. Conteos de IDs ÚNICOS (Individuos/Sesiones)
ids_mch_unicos = df_filtrado[condicion_mch][columna_id_a].nunique()
ids_sanos_unicos = df_filtrado[condicion_sanos][columna_id_a].nunique()

# 7. Conteo de repeticiones (Número de MCH por cada ID)
# Esto nos dice cuántos MCH tiene cada ID de los que son 'Definite'
conteo_repeticiones = df_filtrado[condicion_mch].groupby(columna_id_a).size()

# --- RESULTADOS ---
print(f"--- RESULTADOS DEL FILTRADO ---")
print(f"IDs con MCH 'Definite' (pacientes con hallazgos): {ids_mch_unicos}")
print(f"IDs con TYPE en blanco (presuntos sanos): {ids_sanos_unicos}")
print(f"Total de filas tras la limpieza: {len(df_filtrado)}")

print(f"\n--- DISTRIBUCIÓN DE MCH POR ID")
if not conteo_repeticiones.empty:
    print(conteo_repeticiones.sort_values(ascending=False))
    print(f"\nTotal de filas de MCH (incluyendo repeticiones): {conteo_repeticiones.sum()}")
else:
    print("No se encontraron registros que cumplan la condición de MCH Definite.")

# 8. Análisis de Slice Thickness sobre los filtrados (agregado por mí)
if 'Imaging Protocol' in df_filtrado.columns:
    df_filtrado['Slice_Value'] = df_filtrado['Imaging Protocol'].str.extract(r'Slice Thickness=([0-9.]+)')
    print("\n--- SLICE THICKNESS EN DATASET LIMPIO (Valores más comunes) ---")
    print(df_filtrado['Slice_Value'].value_counts().head(5))


# 9. Guardar resultados
ruta_base = "/media/PORT-DISK/Practicas/MicroBleeds_Generation/"
nombre_archivo = "ADNI_MCH_Clean_Dataset.csv"
ruta_completa = os.path.join(ruta_base, nombre_archivo)

# 2. Guardar el DataFrame filtrado
# Usamos index=False para no añadir una columna de números extra
df_filtrado.to_csv(ruta_completa, index=False)

print(f"Dataset guardado con éxito en: {ruta_completa}")

# En lugar de df_filtrado[condicion_mch], lo correcto para evitar el warning es:
# ids_mch_unicos = df_filtrado.loc[df_filtrado['TYPE'] == 'MCH', columna_id_a].nunique()

Archivos cargados. Columnas en B detectadas: ['Subject ID', 'Phase', 'Sex', 'Weight', 'Research Group', 'Visit', 'Archive Date', 'Study Date', 'Age', 'Modality', 'Description', 'Imaging Protocol', 'Image ID']
--- RESULTADOS DEL FILTRADO ---
IDs con MCH 'Definite' (pacientes con hallazgos): 1812
IDs con TYPE en blanco (presuntos sanos): 3868
Total de filas tras la limpieza: 15242

--- DISTRIBUCIÓN DE MCH POR ID
LONI_IMG_ID
460695     539
399542     525
378882     483
358741     461
342864     458
          ... 
982445       1
982371       1
981037       1
1017752      1
1017739      1
Length: 1812, dtype: int64

Total de filas de MCH (incluyendo repeticiones): 11374

--- SLICE THICKNESS EN DATASET LIMPIO (Valores más comunes) ---
Slice_Value
4.0    15076
5.0      164
4.5        2
Name: count, dtype: int64


/tmp/ipykernel_153267/1665609147.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ids_mch_unicos = df_filtrado[condicion_mch][columna_id_a].nunique()
/tmp/ipykernel_153267/1665609147.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ids_sanos_unicos = df_filtrado[condicion_sanos][columna_id_a].nunique()
/tmp/ipykernel_153267/1665609147.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  conteo_repeticiones = df_filtrado[condicion_mch].groupby(columna_id_a).size()


Dataset guardado con éxito en: /media/PORT-DISK/Practicas/MicroBleeds_Generation/ADNI_MCH_Clean_Dataset.csv
